### Install & Import Packages

In [ ]:
!pip install "flaml[automl]" lightgbm==3.3.1 catboost pygam==0.8.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.2/295.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 67.4 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 4.1.0
    Uninstalling lightgbm-4.1.0:
      Successfully uninstalled lightgbm-4.1.0


In [ ]:
import numpy as np
from numpy import sqrt
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#from category_encoders import OneHotEncoder, MEstimateEncoder, GLMMEncoder, OrdinalEncoder
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, KFold
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor, VotingRegressor, StackingRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.linear_model import PassiveAggressiveRegressor, ARDRegression
from sklearn.linear_model import TheilSenRegressor, RANSACRegressor, HuberRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, roc_auc_score, roc_curve
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform
from pygam import LinearGAM
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

from flaml import AutoML
from flaml.automl.ml import sklearn_metric_loss_score

### Load data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/colab/Regression with a Crab Age Dataset/data/train.csv')
train.shape

train_synthetic = pd.read_csv('/content/drive/MyDrive/colab/Regression with a Crab Age Dataset/data/train_synthetic.csv')
train_synthetic.shape

train = train.append(train_synthetic)

<ipython-input-4-f41fc104bc8b>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train = train.append(train_synthetic)


### Process data

In [ ]:
def process_data(df, drop_id=True):

    # drop id column for train dataset. Need to set to False for test dataset
    if drop_id:
        df.drop('id', axis=1, inplace=True)

    # one hot encode Sex column
    df_dummy = pd.get_dummies(data=df, columns=['Sex'], drop_first=False)

    # label encode Sex column
    # le = LabelEncoder()
    # le.fit(df['Sex'])
    # df['Sex'] = le.transform(df['Sex'])

    # include any other data processing steps here

    return df_dummy

train = process_data(train, drop_id = True)

In [ ]:
def feat_engineer(df):
  # # clean weights by capping overweight values with the total body weights
  # df['Shell Weight'] = np.where(df['Shell Weight'] > df['Weight'], df['Weight'], df['Shell Weight'])
  # df['Viscera Weight'] = np.where(df['Viscera Weight'] > df['Weight'], df['Weight'], df['Viscera Weight'])
  # df['Shucked Weight'] = np.where(df['Shucked Weight'] > df['Weight'], df['Weight'], df['Shucked Weight'])

  # new features
  # df['Viscera Ratio'] = df['Viscera Weight'] / df['Weight']
  # df['Shell Ratio'] = df['Shell Weight'] / df['Weight']
  # df['Shell-to-Body Ratio'] = df['Shell Weight'] / (df['Weight'] + df['Shell Weight'])
  # df['Meat Yield'] = df['Shucked Weight'] / (df['Weight'] + df['Shell Weight'])
  # df['Length-to-Diameter Ratio'] = df['Length'] / df['Diameter']
  # df['Weight-to-VisceraWeight Ratio'] = df['Weight'] / df['Viscera Weight']
  # df['Weight-to-ShellWeight Ratio'] = df['Weight'] / df['Shell Weight']
  # df['Weight-to-ShuckedWeight Ratio'] = df['Weight'] / df['Shucked Weight']

  # df['Surface Area'] = 2 * (df['Length'] * df['Diameter'] + df['Length'] * df['Height'] + df['Diameter'] * df['Height'])
  # df['Volume'] = df['Length'] * df['Diameter'] * df['Height']
  # df['Density'] = df['Weight'] / df['Volume']
  # df['Pseudo BMI'] = df['Weight'] / (df['Height']**2)

  # train['Length^2'] = train['Length']**2
  # train['Diameter^2'] = train['Diameter']**2

  # df['Weight_wo_Viscera'] = df['Shucked Weight'] - df['Viscera Weight']
  # df['Body Condition Index'] = sqrt(df['Length'] * df['Weight'] * df['Shucked Weight'])


feat_engineer(train)

In [ ]:
# Remove outliers based on 'Length' column
train.drop(train[train['Length'] >= 2.0].index, axis=0, inplace=True)
train.drop(train[train['Length'] < 0.35].index, axis=0, inplace=True)

# Remove outliers based on 'Diameter' column
train.drop(train[train['Diameter'] >= 1.6].index, axis=0, inplace=True)

# Remove outliers based on 'Weight' column
train = train.drop(train[train['Weight'] > 77].index)

# Remove outliers based on 'Shucked Weight' column
train = train.drop(train[train['Shucked Weight'] > 40].index)

# Remove outliers based on 'Viscera Weight' column
train = train.drop(train[train['Viscera Weight'] > 17.5].index)

# Remove outliers based on 'Shell Weight' column
train = train.drop(train[train['Shell Weight'] > 25].index)

In [ ]:
train.head()

,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age,Sex_F,Sex_I,Sex_M,Viscera Ratio,Shell Ratio,Shell-to-Body Ratio,Meat Yield,Length-to-Diameter Ratio,Weight-to-VisceraWeight Ratio,Weight-to-ShellWeight Ratio,Weight-to-ShuckedWeight Ratio
0,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9,0,1,0,0.229452,0.288160,0.223699,0.341056,1.297872,4.358209,3.470289,2.276169
1,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8,0,1,0,0.223129,0.326531,0.246154,0.327179,1.333333,4.481707,3.062500,2.304075
2,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9,0,0,1,0.224256,0.268879,0.211903,0.360685,1.247191,4.459184,3.719149,2.185000
3,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11,1,0,0,0.216970,0.296027,0.228411,0.310017,1.203540,4.608925,3.378072,2.488858
4,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8,0,1,0,0.193548,0.255630,0.203587,0.409598,1.234568,5.166667,3.911905,1.944379


### K-Fold Cross Validation

In [ ]:
seed = 42
splits = 10

k = KFold(n_splits = splits, random_state = seed, shuffle = True)

def cross_val_score_with_post(model, cv = k, label = '', include_original = False):

    X = train.copy()
    y = X.pop('Age')

    #initiate prediction arrays and score lists
    val_predictions = np.zeros((len(train)))
    #train_predictions = np.zeros((len(train)))
    train_mae, val_mae = [], []

    #training model, predicting prognosis probability, and evaluating log loss
    for fold, (train_idx, val_idx) in enumerate(cv.split(X, y)):

        #define train set
        X_train = X.iloc[train_idx]
        y_train = y.iloc[train_idx]

        #concat train set with original dataset if include_original is True
        # if include_original == True:
        #     X_train = pd.concat([X_train, encoded_orig_train.drop('Age', axis = 1)])
        #     y_train = pd.concat([y_train, encoded_orig_train['Age']])

        #define validation set
        X_val = X.iloc[val_idx]
        y_val = y.iloc[val_idx]

        #train model
        model.fit(X_train, y_train)

        #make predictions
        train_preds = postprocessor(model.predict(X_train))
        val_preds = postprocessor(model.predict(X_val))

        val_predictions[val_idx] += val_preds

        #evaluate model for a fold
        train_score = mean_absolute_error(y_train, train_preds)
        val_score = mean_absolute_error(y_val, val_preds)

        #append model score for a fold to list
        train_mae.append(train_score)
        val_mae.append(val_score)

    print(f'Val MAE: {np.mean(val_mae):.5f} ± {np.std(val_mae):.5f} | Train MAE: {np.mean(train_mae):.5f} ± {np.std(train_mae):.5f} | {label}')

    return val_mae, val_predictions


def postprocessor(prediction):
    #thanks to @mattop
    unique_targets = np.unique(train['Age'])
    return [min(unique_targets, key = lambda x: abs(x - pred)) for pred in prediction]

In [ ]:
mae_list, oof_list = pd.DataFrame(), pd.DataFrame()

models = [
    #('linear', LinearRegression()),
    #('ridge', Ridge(random_state = seed)),
    #('lasso', Lasso(random_state = seed, max_iter = 1000000)),
    #('elastic', ElasticNet(random_state = seed, max_iter = 1000000)),
    #('huber', HuberRegressor(max_iter = 1000000)),
    #('ard', ARDRegression()),
    #('passive', PassiveAggressiveRegressor(random_state = seed)),
    #('gam', LinearGAM()),
    #('theilsen', TheilSenRegressor(random_state = seed)),
    #('mlp', MLPRegressor(random_state = seed, max_iter = 1000000)),
    #('tab', TabNetRegressor()),
    #('rf', RandomForestRegressor(random_state = seed, criterion = 'absolute_error')),
    ('xgb', XGBRegressor(random_state = seed, objective = 'reg:absoluteerror')),
    ('lgb', LGBMRegressor(random_state = seed, objective = 'mae')),
    #('dart', LGBMRegressor(random_state = seed, boosting_type = 'dart')),
    ('cb', CatBoostRegressor(random_state = seed, objective = 'MAE', verbose = 0)),
    #('gb', GradientBoostingRegressor(random_state = seed, loss = 'absolute_error')),
    #('hgb', HistGradientBoostingRegressor(random_state = seed, loss = 'absolute_error'))
    #('knn', KNeighborsRegressor())
]

for (label, model) in models:
     mae_list[label], oof_list[label] = cross_val_score_with_post(
         make_pipeline(StandardScaler(), model),
         label = label
     )

Val MAE: 1.30494 ± 0.00825 | Train MAE: 1.28364 ± 0.00412 | xgb
Val MAE: 1.32285 ± 0.01080 | Train MAE: 1.31688 ± 0.00241 | lgb
Val MAE: 1.36663 ± 0.01005 | Train MAE: 1.36003 ± 0.00135 | dart
Val MAE: 1.30614 ± 0.01209 | Train MAE: 1.29705 ± 0.00187 | cb
Val MAE: 1.37768 ± 0.01649 | Train MAE: 1.37553 ± 0.01008 | gb
Val MAE: 1.32257 ± 0.01051 | Train MAE: 1.31554 ± 0.00335 | hgb


In [ ]:
ensemble_models = [
    ('xgb', XGBRegressor(random_state = seed, objective = 'reg:absoluteerror', n_estimators = 2500)),
    ('cb', CatBoostRegressor(random_state = seed, objective = 'MAE', verbose = 0, n_estimators = 2500)),
]

voter = VotingRegressor(ensemble_models)

_ = cross_val_score_with_post(make_pipeline(StandardScaler(), voter), label = 'Voting Ensemble')

Val MAE: 1.29621 ± 0.01075 | Train MAE: 1.27000 ± 0.00295 | Voting Ensemble


### Make predictions on the submission data and output as csv


In [ ]:
test = process_data(pd.read_csv('/content/drive/MyDrive/colab/Regression with a Crab Age Dataset/data/test.csv'), drop_id = False)
#feat_engineer(test)

X = train.copy()
y = X.pop('Age')

prediction = postprocessor(voter.fit(X, y).predict(test.drop('id', axis=1)))

test['Age'] = prediction

# select only id and predictions column
output = test[['id', 'Age']]

# output as csv
output.to_csv('submission.csv', index=False)

# Hyperparameter tuning

In [ ]:
param_grid = {
    'alpha': [1, 0, 0.1, 0.01],
    'solver': ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg']
             }

ridge = Ridge()
gs = GridSearchCV(ridge, # specify the model
                  param_grid=param_grid, # hyperparameters to test
                  scoring='neg_mean_absolute_error', # values are negative but we can just look at the absolute value
                  cv=5, # number of folds in the cross validation
                  verbose=2) # amount of info to print during training. 0 = print nothing, higher = print more
gs.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV] alpha=1, solver=auto ............................................
[CV] ............................. alpha=1, solver=auto, total=   0.0s
[CV] alpha=1, solver=auto ............................................
[CV] ............................. alpha=1, solver=auto, total=   0.0s
[CV] alpha=1, solver=auto ............................................
[CV] ............................. alpha=1, solver=auto, total=   0.0s
[CV] alpha=1, solver=auto ............................................
[CV] ............................. alpha=1, solver=auto, total=   0.0s
[CV] alpha=1, solver=auto ............................................
[CV] ............................. alpha=1, solver=auto, total=   0.0s
[CV] alpha=1, solver=svd .............................................
[CV] .............................. alpha=1, solver=svd, total=   0.0s
[CV] alpha=1, solver=svd .............................................
[CV] ..........

[CV] ....................... alpha=0.1, solver=cholesky, total=   0.0s
[CV] alpha=0.1, solver=cholesky ......................................
[CV] ....................... alpha=0.1, solver=cholesky, total=   0.0s
[CV] alpha=0.1, solver=lsqr ..........................................
[CV] ........................... alpha=0.1, solver=lsqr, total=   0.0s
[CV] alpha=0.1, solver=lsqr ..........................................
[CV] ........................... alpha=0.1, solver=lsqr, total=   0.0s
[CV] alpha=0.1, solver=lsqr ..........................................
[CV] ........................... alpha=0.1, solver=lsqr, total=   0.0s
[CV] alpha=0.1, solver=lsqr ..........................................
[CV] ........................... alpha=0.1, solver=lsqr, total=   0.0s
[CV] alpha=0.1, solver=lsqr ..........................................
[CV] ........................... alpha=0.1, solver=lsqr, total=   0.0s
[CV] alpha=0.1, solver=sparse_cg .....................................
[CV] .

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.1s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Ridge(alpha=1.0, copy_X=True, fit_intercept=True,
                             max_iter=None, normalize=False, random_state=None,
                             solver='auto', tol=0.001),
             iid='warn', n_jobs=None,
             param_grid={'alpha': [1, 0, 0.1, 0.01],
                         'solver': ['auto', 'svd', 'cholesky', 'lsqr',
                                    'sparse_cg']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_absolute_error', verbose=2)

In [ ]:
gs.best_params_

{'alpha': 0, 'solver': 'auto'}